# MIT 6.006 Part 2

This week I will finish the introduction to algorithms course I started last week.

## Single-Source Shortest Path Problem

Often we want to find the shortest distance from one vertex to another. This problem can be hard to solve as even with a relatively small number of vertices it would take a long time (many years) to find the shortest path to a given vertex.

If a graph is weighted it means there is a cost to go from one vertex to another.

A graph can have negative weights. One example is when a taxi gets from one destination to another a negative weight could represent money earned and a positive weight could represent money spent on fuel.

If a graph has a negative cycle then certain shortest paths are undefined.

Single-Source Shortest Path algorithms work by setting the starting node to have a cost of 0 and all other nodes to having a cost of infinity.

Then we select an edge (u, v).

If d[v] > d[u] + w(u, v) then we update d[v] to d[u] + w(u, v). We also set π[v] = u. 

Where d is a list of all the distances to the other nodes and π is a list of all the parents of the other nodes.

We repeat this until all d[v] <= d[u] + w(u, v)

Hence this means that d has converged to the actual shortest path from one vertex to another.

### Optimal Substructure

Theorem: Subpaths of shortest paths are shortest paths.

### Triangle Inequality

Theorem: For all u, v, x ∈ X we have:

δ(u,v)<= δ(u,x) +δ(x,v)

## Dijikstra's Algorithm

Only works for graphs in which every edge has a positive weight.

The algorithm finds the shortest path from a given vertex to every other vertex in the graph.

### The Algorithm

1. Set the weight of the starting vertex to 0 and all other vertices to infinity and add the distances to a priority queue.
2. Pop from the priority queue and assign to variable v.
3. Update the weight of adjacent nodes to the vertex v if the cost is lower.
4. Repeat 2-3 until the priority queue is empty.

![dijikstra's algorithm](https://i.stack.imgur.com/90Qwu.png)

Using the above graph as an example I will find the shortest path from A to F.

In [43]:
import heapq

'''
Represents a single graph.
'''        
class Graph:
    def __init__(self, *nodes):
        # Creating the adjacency list.
        self.graph = {node:{} for node in nodes}
        
    def add_edge(self, u, v, weight):
        self.graph[u][v] = weight
        
g = Graph('A', 'B', 'C', 'D', 'E', 'F')
g.add_edge('A', 'B', 3)
g.add_edge('A', 'C', 5)
g.add_edge('A', 'D', 9)
g.add_edge('B', 'A', 3)
g.add_edge('C', 'A', 5)
g.add_edge('D', 'A', 9)
g.add_edge('B', 'C', 3)
g.add_edge('B', 'E', 7)
g.add_edge('B', 'D', 4)
g.add_edge('C', 'B', 3)
g.add_edge('C', 'D', 2)
g.add_edge('C', 'E', 6)
g.add_edge('C', 'F', 8)
g.add_edge('D', 'C', 2)
g.add_edge('D', 'B', 4)
g.add_edge('D', 'E', 2)
g.add_edge('D', 'F', 2)
g.add_edge('E', 'D', 2)
g.add_edge('E', 'C', 6)
g.add_edge('E', 'B', 7)
g.add_edge('E', 'F', 5)
g.add_edge('F', 'D', 2)
g.add_edge('F', 'E', 2)
g.add_edge('F', 'C', 8)

'''
Implementing Dijikstra's Algorithm in Python
'''
def dijikstra(gr, start_point):
    # Keeps track of all the distances.
    distances = {vertex:{'distance': float('inf'), 'prev_node': None} for vertex in g.graph.keys()}
    distances[start_point]['distance'] = 0
    
    remaining_nodes = distances.copy()
    
    priority_queue = [(value['distance'], key) for key, value in distances.items()]
    
    # Convert the priority queue to a heap
    heapq.heapify(priority_queue)
    
    visited_nodes = set()
    
    # Add the start point to the queue
    # Get the last element in the priority queue.
    while (len(priority_queue) != 0):
        current_node = heapq.heappop(priority_queue)
        
        # Update the values of other nodes
        connected_nodes = gr.graph[current_node[1]].items()
        
        for node, weight in connected_nodes:
            if node not in visited_nodes and distances[node]['distance'] > current_node[0] + weight:
                distances[node]['distance'] = current_node[0] + weight
                distances[node]['prev_node'] = current_node[1]
                
        visited_nodes.add(current_node[1])
        

        del remaining_nodes[current_node[1]]
        
        priority_queue = [(value['distance'], key) for key, value in remaining_nodes.items()]
        # Convert the priority queue to a heap
        heapq.heapify(priority_queue)
    
    return distances
        
shortest_distances = dijikstra(g, 'A')

for node, info in shortest_distances.items():
    print(node, info)

A {'distance': 0, 'prev_node': None}
B {'distance': 3, 'prev_node': 'A'}
C {'distance': 5, 'prev_node': 'A'}
D {'distance': 7, 'prev_node': 'B'}
E {'distance': 9, 'prev_node': 'D'}
F {'distance': 9, 'prev_node': 'D'}


## Optimizing Dijikstra's Algorithm

Dijikstra's algorithm can be optimized so that in practice it runs a lot faster. 

The speedup techniques below will not change worst-case behaviour. But causes the algorithm to speed up in practice by reducing the number of vertices visited.
### Single Source Single Target

One way to improve the performance of Dijikstra's algorithm is to stop when we have visited the destination node. At this point we can stop as we do not need to know the most efficient route to the other nodes.

### Bi-Directional Search

Using a single target destination we can speed up Dijikstra's algorithm even further. 

In bi-directional search we alternate search from source and backward searching from the destination.
When we remove an element from the queue that has already been visited by the other search then we stop.
When we stop we find the node x which has a minimum value of df(x) + db(x).

In [104]:
# Implementing Bi-Directional Search
def bi_directional_search(gr, source, destination):
    forward_distances = {vertex:{'distance': float('inf'), 'prev_node': None} for vertex in g.graph.keys()}
    backward_distances = {vertex:{'distance': float('inf'), 'prev_node': None} for vertex in g.graph.keys()}
    
    forward_distances[source]['distance'] = 0
    backward_distances[destination]['distance'] = 0
    
    remaining_forward = forward_distances.copy()
    remaining_backward = backward_distances.copy()
    
    forward_priority_queue = [(value['distance'], key) for key, value in forward_distances.items()]
    backward_priority_queue = [(value['distance'], key) for key, value in backward_distances.items()]
    
    heapq.heapify(forward_priority_queue)
    heapq.heapify(backward_priority_queue)
    
    visited_forward = set()
    visited_backward = set()
    
    done = False
    
    is_forward = True
    
    while not done:
        current_node = heapq.heappop(forward_priority_queue) if is_forward else heapq.heappop(backward_priority_queue)
        
        if current_node[1] in visited_forward or current_node[1] in visited_backward:
            break
            
        # Get adjacent nodes.
        connected_nodes = gr.graph[current_node[1]].items()
                    
        distances = forward_distances if is_forward else backward_distances
        
        for node, weight in connected_nodes:
            if distances[node]['distance'] > current_node[0] + weight:
                distances[node]['distance'] = current_node[0] + weight
                distances[node]['prev_node'] = current_node[1]
                
        if is_forward: 
            del remaining_forward[current_node[1]]
            heapq.heapify(forward_priority_queue)
            visited_forward.add(current_node[1])
            forward_priority_queue = [(value['distance'], key) for key, value in remaining_forward.items()]
            heapq.heapify(forward_priority_queue)
        else:
            del remaining_backward[current_node[1]]
            heapq.heapify(backward_priority_queue)
            visited_backward.add(current_node[1])
            backward_priority_queue = [(value['distance'], key) for key, value in remaining_backward.items()]
            heapq.heapify(backward_priority_queue)
            
        is_forward = not is_forward
        
        
    lowest_distance = float('inf')
    common_node = None

    common_nodes = [item for item in forward_distances.items() if backward_distances[item[0]]['distance'] != float('inf')
                                                                    and item[1]['distance'] != float('inf')]
                    
    for item in common_nodes:
        if forward_distances[item[0]]['distance'] + backward_distances[item[0]]['distance'] < lowest_distance:
            lowest_distance = forward_distances[item[0]]['distance'] + backward_distances[item[0]]['distance']
            common_node = item[0]
            
    # Find the path
    path = []
    
    path.insert(0, common_node)
    
    current_node = common_node
    while current_node != source:
        current_node = forward_distances[current_node]['prev_node']
        
        path.insert(0, current_node)
        
    current_node = common_node
    while current_node != destination:
        current_node = backward_distances[current_node]['prev_node']
        path.append(current_node)
                
    return path, lowest_distance
                
bi_directional_search(g, 'A', 'F')                        

(['A', 'B', 'D', 'F'], 9)

### Goal-Directed Search (A* Algorithm)

In a goal directed search we sort the priority queue by an additional heuristic function to make it more likely that the algorithm picks a vertex which is part of the final path to the destination thus speeding up computation in practice.

There are different heuristics that we can use.

#### Manhatten Distance

h = abs (current_cell.x - goal.x) + abs( current_cell.y - goal.y)

#### Diagonal Distance

h = max { abs (current_cell.x - goal.x), abs (current_cell.y - goal.y) }

#### Euclidean Distance

h = sqrt( (current_cell.x - goal.x) ^ 2 + (current_cell.y - goal.y)^2 )

These are just a few examples of heuristic functions.

### Skipton To St Andrews

I will attempt to find the best route from Skipton to St Andrews using this highly abstracted graph I have created.

![skipton graph](https://tim-beatham.github.io/Week8/skipton_to_st_andrews.png)

Each node represents a major town/city that we may need to pass to get to St Andrews.

- S = Skipton
- B = Bradford
- L = Leeds
- N = Newcastle
- E = Edinburgh 
- K = Kendal
- C = Carlisle
- G = Glasgow
- St = Stirling
- SA = St Andrews

Additionally I will the euclidean heuristic distance using longitude and latitude coordinates to help find the optimum route.

The weights on the graph represents the time it takes to travel from one node to the other in minutes. The weights are not the distances between the two nodes.

In [195]:
'''
An implementation of the A* algorithm.
Find the best way to get from Skipton
to St Andrews.
'''
class Vertex:
    def __init__(self, name, longitude, latitude):
        self.name = name
        self.longitude = longitude
        self.latitude = latitude
    
    def __hash__(self):
        return hash((self.name, self.longitude, self.latitude))
    
    def __eq__(self, other):
        return (self.name, self.longitude, self.latitude) == (other.name, other.longitude, other.latitude)
    
    def __repr__(self):
        return "{0}({1}, {2})".format(self.name, self.longitude, self.latitude)
    
skipton = Vertex("Skipton", 53.9628, 2.0163)
bradford = Vertex("Bradford", 53.7960, 1.7594)
leeds = Vertex("Leeds", 53.8008, 1.5491)
newcastle = Vertex("Newcastle", 54.9783, 1.6178)
kendal = Vertex("Kendal", 54.3280, 2.7463)
carlisle = Vertex("Carlisle", 54.8925, 2.9329)  
glasgow = Vertex("Glasgow", 55.8642, 4.2518)
edinburgh = Vertex("Edinburgh", 55.9533, 3.1883)
st_andrews = Vertex("St Andrews", 56.3398, 2.7967)
stirling = Vertex("Stirling", 56.1165, 3.9369)
sheffield = Vertex("Sheffield", 53.3811, 1.4701)
manchester = Vertex("Manchester", 53.3811, 1.4701)

travel_graph = Graph(skipton, bradford, leeds, newcastle, kendal, carlisle, glasgow, edinburgh, st_andrews,
                    stirling, sheffield, manchester)

travel_graph.add_edge(skipton, bradford, 35)
travel_graph.add_edge(bradford, skipton, 35)
travel_graph.add_edge(skipton, leeds, 58)
travel_graph.add_edge(leeds, skipton, 58)
travel_graph.add_edge(skipton, kendal, 71)
travel_graph.add_edge(kendal, skipton, 71)
travel_graph.add_edge(bradford, leeds, 26)
travel_graph.add_edge(leeds, bradford, 26)
travel_graph.add_edge(kendal, carlisle, 61)
travel_graph.add_edge(carlisle, kendal, 61)
travel_graph.add_edge(carlisle, glasgow, 98)
travel_graph.add_edge(glasgow, carlisle, 98)
travel_graph.add_edge(edinburgh, glasgow, 63)
travel_graph.add_edge(glasgow, edinburgh, 63)
travel_graph.add_edge(glasgow, stirling, 37)
travel_graph.add_edge(stirling, glasgow, 37)
travel_graph.add_edge(stirling, st_andrews, 86)
travel_graph.add_edge(st_andrews, stirling, 86)
travel_graph.add_edge(edinburgh, st_andrews, 83)
travel_graph.add_edge(st_andrews, edinburgh, 83)
travel_graph.add_edge(leeds, newcastle, 108)
travel_graph.add_edge(newcastle, leeds, 108)
travel_graph.add_edge(newcastle, edinburgh, 144)
travel_graph.add_edge(edinburgh, newcastle, 144)
travel_graph.add_edge(skipton, manchester, 79)
travel_graph.add_edge(manchester, skipton, 79)
travel_graph.add_edge(skipton, sheffield, 104)
travel_graph.add_edge(sheffield, skipton, 104)
travel_graph.add_edge(manchester, sheffield, 89)
travel_graph.add_edge(sheffield, manchester, 89)

from math import sqrt

heuristic_factor = 100

def euclidean_distance(node1, node2):
    return sqrt((node1.longitude - node2.longitude) ** 2 + (node1.latitude - node2.latitude) ** 2) \
                                                        * heuristic_factor

def a_star(g, start_vertex, end_vertex):
    distances = {vertex:{'distance': float('inf'), 'prev_node': None} for vertex in g.graph.keys()}
    distances[start_vertex]['distance'] = 0
    
    remaining_nodes = sorted(list(distances.keys()), key=lambda vertex: distances[vertex]['distance'])
    
    current_node = remaining_nodes.pop(0)
    
    visited = set()
    
    while current_node != end_vertex:
        
        visited.add(current_node)


        # Get the surrounding elements.
        for node, weight in g.graph[current_node].items():
            # Update the distance
            if node not in visited and distances[current_node]['distance'] + weight < distances[node]['distance']:
                distances[node]['distance'] = distances[current_node]['distance'] + weight
                distances[node]['prev_node'] = current_node
                        
        remaining_nodes = sorted(remaining_nodes, key=lambda vertex: distances[vertex]['distance'] 
                                 + euclidean_distance(vertex, end_vertex))
        
        current_node = remaining_nodes.pop(1)
        
    total_cost = distances[end_vertex]['distance']
    
    current_node = end_vertex
    
    path = []
    
    while current_node != start_vertex:
        path.insert(0, current_node)
        current_node = distances[current_node]['prev_node']
        
    print("Distances", distances)
    print()
    print("Visited", visited)
    
        
    return total_cost, path
        
a_star(travel_graph, skipton, st_andrews)

Distances {Skipton(53.9628, 2.0163): {'distance': 0, 'prev_node': None}, Bradford(53.796, 1.7594): {'distance': 35, 'prev_node': Skipton(53.9628, 2.0163)}, Leeds(53.8008, 1.5491): {'distance': 58, 'prev_node': Skipton(53.9628, 2.0163)}, Newcastle(54.9783, 1.6178): {'distance': 166, 'prev_node': Leeds(53.8008, 1.5491)}, Kendal(54.328, 2.7463): {'distance': 71, 'prev_node': Skipton(53.9628, 2.0163)}, Carlisle(54.8925, 2.9329): {'distance': inf, 'prev_node': None}, Glasgow(55.8642, 4.2518): {'distance': 373, 'prev_node': Edinburgh(55.9533, 3.1883)}, Edinburgh(55.9533, 3.1883): {'distance': 310, 'prev_node': Newcastle(54.9783, 1.6178)}, St Andrews(56.3398, 2.7967): {'distance': 393, 'prev_node': Edinburgh(55.9533, 3.1883)}, Stirling(56.1165, 3.9369): {'distance': inf, 'prev_node': None}, Sheffield(53.3811, 1.4701): {'distance': 104, 'prev_node': Skipton(53.9628, 2.0163)}, Manchester(53.3811, 1.4701): {'distance': 79, 'prev_node': Skipton(53.9628, 2.0163)}}

Visited {Leeds(53.8008, 1.5491),

(393,
 [Leeds(53.8008, 1.5491),
  Newcastle(54.9783, 1.6178),
  Edinburgh(55.9533, 3.1883),
  St Andrews(56.3398, 2.7967)])

### The Problem With Using Euclidean In The Above Example

Using euclidean distance as a heuristic has many problems in the above example which shows it is not the best heuristic for the problem.

If I change the heuristic factor too much then the algorithm always goes in the direction which is closest to St Andrews as the bird flies. Hence when the heuristic_factor variable is 100 the algorithm says that the quickest route to St Andrews is Skipton -> Leeds -> Newcastle -> Edinburgh -> St Andrews. When in fact the quiest route is Skipton -> Kendal -> Carlisle -> Glasgow -> Stirling -> St Andrews

When the heuristic factor is too low the algorithm is not influenced greatly and explores nodes it does not need to such as Manchester and Sheffield. It does however ultimately find the shortest path.

## Bellman-Ford Algorithm

Dijikstra's algorithm only works for graphs that have positive weights. Dijikstra's algorithm does not work for graphs with negative weights.

This is because a graph that has a negative cycle has an undefined shortest path as when you keep iterating over this cycle the total cost gets smaller and smaller.

The algorithm is less efficient than Dijikstra's algorithm however it runs in O(VE) time whereas Dijikstras algorithm runs in O ( V + E l o g V ) if you use a min-heap data structure as the priority queue.

If the graph is simple then it is likely that E = O(V^2) and therefore the Bellman-Ford algorithm is comparable to O(V ^ 3) in some cases.


![bellman ford graph](https://java2blog.com/wp-content/uploads/2018/11/bellman-Ford1.jpg)

In [223]:
bellman_ford_graph = Graph('1', '2', '3', '4', '5')

bellman_ford_graph.add_edge('1', '3', 1)
bellman_ford_graph.add_edge('1', '2', 2)
bellman_ford_graph.add_edge('2', '4', 1)
bellman_ford_graph.add_edge('4', '1', 5)
bellman_ford_graph.add_edge('3', '4', 3)
bellman_ford_graph.add_edge('1', '5', -5)
bellman_ford_graph.add_edge('5', '2', -6)

'''
Implementing the Bellman-Ford algorithm
'''
def bellman_ford(g, start_point):
    distances = {vertex:{'distance': float('inf'), 'prev_node': None} for vertex in g.graph.keys()}
    distances[start_point]['distance'] = 0
    
    
    # Iterate over every vertex in the graph.
    for i in range(len(distances.items())):
        # Iterate over every edge in the graph
        for source_vertex, edges in g.graph.items():
            for dest_vertex, weight in edges.items():
                temp_distance = distances[source_vertex]['distance'] + weight
                if temp_distance < distances[dest_vertex]['distance']:
                    distances[dest_vertex]['distance'] = temp_distance
                    distances[dest_vertex]['prev_node'] = source_vertex
                    
                                        
    for source_vertex, edges in g.graph.items():
        for dest_vertex, weight in edges.items():
            if distances[source_vertex]['distance'] + weight < distances[dest_vertex]['distance']:\
                return "Negative Cycle Exists"
    
    return distances

print(bellman_ford(bellman_ford_graph, '1'))
print()

final_travel = bellman_ford(travel_graph, skipton)

for node, info in final_travel.items():
    print(node, info)

Negative Cycle Exists

Skipton(53.9628, 2.0163) {'distance': 0, 'prev_node': None}
Bradford(53.796, 1.7594) {'distance': 35, 'prev_node': Skipton(53.9628, 2.0163)}
Leeds(53.8008, 1.5491) {'distance': 58, 'prev_node': Skipton(53.9628, 2.0163)}
Newcastle(54.9783, 1.6178) {'distance': 166, 'prev_node': Leeds(53.8008, 1.5491)}
Kendal(54.328, 2.7463) {'distance': 71, 'prev_node': Skipton(53.9628, 2.0163)}
Carlisle(54.8925, 2.9329) {'distance': 132, 'prev_node': Kendal(54.328, 2.7463)}
Glasgow(55.8642, 4.2518) {'distance': 230, 'prev_node': Carlisle(54.8925, 2.9329)}
Edinburgh(55.9533, 3.1883) {'distance': 293, 'prev_node': Glasgow(55.8642, 4.2518)}
St Andrews(56.3398, 2.7967) {'distance': 353, 'prev_node': Stirling(56.1165, 3.9369)}
Stirling(56.1165, 3.9369) {'distance': 267, 'prev_node': Glasgow(55.8642, 4.2518)}
Sheffield(53.3811, 1.4701) {'distance': 104, 'prev_node': Skipton(53.9628, 2.0163)}
Manchester(53.3811, 1.4701) {'distance': 79, 'prev_node': Skipton(53.9628, 2.0163)}


## Dynamic Programming

In [222]:
# Explanation of some dynamic programming.